In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import re
import time

In [2]:
#You can implement the headless version for Chrome webdriver, but I really enjoy knowing what's going on in the browser
driver = webdriver.Chrome()
driver.get('https://search.wdoms.org/')
html = BeautifulSoup(driver.page_source, 'html.parser')

#Constants
##Each results page displays 20 schools/rows
results_per_page = 20

##Empty Dataframe
schools_df = pd.DataFrame()

##DataFrame Index Initialization
index_counter = 0

def get_number_of_clicks(html):
    #In case the results page is empty you return number_of_clicks = 0
    try:
        number_of_results = html.find('div',{'class':'span4 pagination'}).text
        number_of_results = int(re.search('\d+',number_of_results)[0])
    except:
        return 0
    if number_of_results%20 !=0:
        number_of_pages = int(((number_of_results-(number_of_results%20))/20)+1)
    else:
        number_of_pages = int(((number_of_results-(number_of_results%20))/20))
    return number_of_pages

def get_schools_info(html,schools_df,index_counter):
    #Using BeautifulSoup to extract schools information
    table_results = html.find('table',{'id':'tblResult'}).findAll('tr')
    for row in range(1,len(table_results)):
        country = table_results[row].findAll('td')[1].text
        school_name = table_results[row].findAll('td')[2].find('a').text
        city_name = table_results[row].findAll('td')[3].text
        index_counter+=1
        #Appending extracted information to DataFrame schools_df
        schools_df.loc[index_counter,'COUNTRY'] = country
        schools_df.loc[index_counter,'SCHOOL NAME'] = school_name
        schools_df.loc[index_counter,'CITY NAME'] = city_name
    return index_counter
     
for country_id in range(2,183):
    #Fills up the form and click the search button
    driver.find_element_by_xpath('//*[@id="countrylist"]/option['+str(country_id)+']').click()
    driver.find_element_by_xpath('//*[@id="operationflag"]/option[2]').click()
    driver.find_element_by_xpath('//*[@id="submit"]').click()
    time.sleep(2)
    
    #Get the html from Results Page Number 1
    html = BeautifulSoup(driver.page_source, 'html.parser')
        
    #Calculates how many clicks you need to make to the "next" button
    number_of_clicks=get_number_of_clicks(html)
    if number_of_clicks != 0:
        for page in range(0,number_of_clicks):
            if page>0:
                driver.find_element_by_xpath('//*[@id="Next"]').click()
                time.sleep(1)
                html = BeautifulSoup(driver.page_source, 'html.parser')
                index_counter = get_schools_info(html,schools_df,index_counter)
            else:
                index_counter = get_schools_info(html,schools_df,index_counter)  

schools_df.to_csv('WorldSchoolsNames2020.csv')   